## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
file_to_load ="C:\\Users\\alice\\Desktop\\BootCamp_DataAnaysis\\Practice_and_Challenge\\WeatherPy\\Weather_Database\\WeatherPy_Database.csv"

city_data_df = pd.read_csv(file_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Grand Gaube,MU,-20.0064,57.6608,77.00,79,100,8.99,overcast clouds
1,1,Hazorasp,UZ,41.3194,61.0742,60.80,31,0,11.50,clear sky
2,2,Bluff,NZ,-46.6000,168.3333,52.00,94,100,20.56,overcast clouds
3,3,Camaná,PE,-16.6228,-72.7111,67.73,86,69,8.23,broken clouds
4,4,Ponta do Sol,PT,32.6667,-17.1000,60.01,83,69,5.99,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Grand Gaube,MU,-20.0064,57.6608,77.00,79,100,8.99,overcast clouds
10,10,Acaraú,BR,-2.8856,-40.1200,75.92,88,21,5.21,few clouds
13,13,Bontang,ID,0.1333,117.5000,86.27,66,79,9.73,broken clouds
14,14,Arraial do Cabo,BR,-22.9661,-42.0278,73.11,77,7,22.03,clear sky
16,16,Vaini,TO,-21.2000,-175.2000,82.40,89,40,4.61,light rain
17,17,Avarua,CK,-21.2078,-159.7750,80.60,74,75,17.27,broken clouds
18,18,Yining,CN,29.0361,114.5636,78.89,61,100,2.30,overcast clouds
19,19,Malindi,KE,-3.2175,40.1191,78.80,94,75,14.90,broken clouds
20,20,Puerto Ayora,EC,-0.7393,-90.3518,75.00,95,46,1.01,scattered clouds
21,21,Bima,ID,-8.4667,118.7167,88.45,54,34,0.29,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                225
City                   225
Country                225
Lat                    225
Lng                    225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Current Description    225
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how="any")
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Grand Gaube,MU,-20.0064,57.6608,77.00,79,100,8.99,overcast clouds
10,10,Acaraú,BR,-2.8856,-40.1200,75.92,88,21,5.21,few clouds
13,13,Bontang,ID,0.1333,117.5000,86.27,66,79,9.73,broken clouds
14,14,Arraial do Cabo,BR,-22.9661,-42.0278,73.11,77,7,22.03,clear sky
16,16,Vaini,TO,-21.2000,-175.2000,82.40,89,40,4.61,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Grand Gaube,MU,77.00,overcast clouds,-20.0064,57.6608,
10,Acaraú,BR,75.92,few clouds,-2.8856,-40.1200,
13,Bontang,ID,86.27,broken clouds,0.1333,117.5000,
14,Arraial do Cabo,BR,73.11,clear sky,-22.9661,-42.0278,
16,Vaini,TO,82.40,light rain,-21.2000,-175.2000,
17,Avarua,CK,80.60,broken clouds,-21.2078,-159.7750,
18,Yining,CN,78.89,overcast clouds,29.0361,114.5636,
19,Malindi,KE,78.80,broken clouds,-3.2175,40.1191,
20,Puerto Ayora,EC,75.00,scattered clouds,-0.7393,-90.3518,
21,Bima,ID,88.45,scattered clouds,-8.4667,118.7167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"  
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Grand Gaube,MU,77.00,overcast clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
10,Acaraú,BR,75.92,few clouds,-2.8856,-40.1200,castelo pizzaria
13,Bontang,ID,86.27,broken clouds,0.1333,117.5000,KOPKAR Hotel Bintang Sintuk
14,Arraial do Cabo,BR,73.11,clear sky,-22.9661,-42.0278,Pousada Porto Praia
16,Vaini,TO,82.40,light rain,-21.2000,-175.2000,Keleti Beach Resort
17,Avarua,CK,80.60,broken clouds,-21.2078,-159.7750,Paradise Inn
18,Yining,CN,78.89,overcast clouds,29.0361,114.5636,Xiushui Hotel
19,Malindi,KE,78.80,broken clouds,-3.2175,40.1191,African House Resort
20,Puerto Ayora,EC,75.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
21,Bima,ID,88.45,scattered clouds,-8.4667,118.7167,Lambitu Hotel


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head(20)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Grand Gaube,MU,77.00,overcast clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
10,Acaraú,BR,75.92,few clouds,-2.8856,-40.1200,castelo pizzaria
13,Bontang,ID,86.27,broken clouds,0.1333,117.5000,KOPKAR Hotel Bintang Sintuk
14,Arraial do Cabo,BR,73.11,clear sky,-22.9661,-42.0278,Pousada Porto Praia
16,Vaini,TO,82.40,light rain,-21.2000,-175.2000,Keleti Beach Resort
17,Avarua,CK,80.60,broken clouds,-21.2078,-159.7750,Paradise Inn
18,Yining,CN,78.89,overcast clouds,29.0361,114.5636,Xiushui Hotel
19,Malindi,KE,78.80,broken clouds,-3.2175,40.1191,African House Resort
20,Puerto Ayora,EC,75.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
21,Bima,ID,88.45,scattered clouds,-8.4667,118.7167,Lambitu Hotel


In [11]:
clean_hotel_df.count()

City                   225
Country                225
Max Temp               225
Current Description    225
Lat                    225
Lng                    225
Hotel Name             225
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
locations.head()

,Lat,Lng
0,-20.0064,57.6608
10,-2.8856,-40.1200
13,0.1333,117.5000
14,-22.9661,-42.0278
16,-21.2000,-175.2000


In [15]:
# print(hotel_info)

In [16]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))